# Análise dos dados e transformações necessárias, além de limpeza

In [0]:
from pyspark.sql.functions import isnan, when, array_distinct, split

> Visualizar tipos

In [0]:
df_goodreads = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/bronze/goodreads')
df_kindle = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/bronze/kindle')

In [0]:
df_goodreads.toPandas().head(2)

In [0]:
df_kindle.toPandas().head(2)

## Exclusão das colunas que não serão utilizadas

In [0]:
df_goodreads = df_goodreads.drop(df_goodreads.desc, df_goodreads.isbn13, df_goodreads.link)

## Verificação de nulos

### Dos livros do GoodReads

In [0]:
print(f'Quantidade de autores nulos: {df_goodreads.filter(df_goodreads.author.isNull()).count()}')
print(f'Quantidade de formatos de livros nulos: {df_goodreads.filter(df_goodreads.bookformat.isNull()).count()}')
print(f'Quantidade de gêneros nulos: {df_goodreads.filter(df_goodreads.genre.isNull()).count()}')
print(f'Quantidade de livros sem imagens: {df_goodreads.filter(df_goodreads.img.isNull()).count()}')
print(f'Quantidade de isbn nulos: {df_goodreads.filter(df_goodreads.isbn.isNull()).count()}')
print(f'Quantidade de páginas nan: {df_goodreads.filter(isnan(df_goodreads.pages)).count()}')
print(f'Quantidade de classificações nan: {df_goodreads.filter(isnan(df_goodreads.rating)).count()}')
print(f'Quantidade de análises nan: {df_goodreads.filter(isnan(df_goodreads.reviews)).count()}')
print(f'Quantidade de títulos nulos: {df_goodreads.filter(df_goodreads.author.isNull()).count()}')
print(f'Quantidade de total classificações nan: {df_goodreads.filter(isnan(df_goodreads.totalratings)).count()}')

> Modificar de nulo para um texto mais coerente

> Coluna _bookformat_

In [0]:
df_goodreads = df_goodreads.withColumn('bookformat',
                        when(df_goodreads.bookformat.isNull(), "Unknown")
                        .otherwise(df_goodreads.bookformat))

df_goodreads.where(df_goodreads.bookformat == "Unknown").show(2)

> Coluna _genre_

In [0]:
df_goodreads = df_goodreads.withColumn('genre',
                        when(df_goodreads.genre.isNull(), "Unknown")
                        .otherwise(df_goodreads.genre))

df_goodreads.where(df_goodreads.genre == "Unknown").show(2)

> Coluna _img_

In [0]:
df_goodreads = df_goodreads.withColumn('img',
                        when(df_goodreads.img.isNull(), "Unknown")
                        .otherwise(df_goodreads.img))

df_goodreads.where(df_goodreads.img == "Unknown").show(2)

> Coluna _isbn_

In [0]:
df_goodreads = df_goodreads.withColumn('isbn',
                        when(df_goodreads.isbn.isNull(), "Unknown")
                        .otherwise(df_goodreads.isbn))

df_goodreads.where(df_goodreads.isbn == "Unknown").show(2)

### Dos livros do Kindle

In [0]:
print(f'Quantidade de asin nulos: {df_kindle.filter(df_kindle.asin.isNull()).count()}')
print(f'Quantidade de títulos nulos: {df_kindle.filter(df_kindle.title.isNull()).count()}')
print(f'Quantidade de autor nulos: {df_kindle.filter(df_kindle.author.isNull()).count()}')
print(f'Quantidade de vendas nulas: {df_kindle.filter(df_kindle.soldBy.isNull()).count()}')
print(f'Quantidade de imagens nulas: {df_kindle.filter(df_kindle.imgUrl.isNull()).count()}')
print(f'Quantidade de urls de produtos nulos: {df_kindle.filter(df_kindle.productURL.isNull()).count()}')
print(f'Quantidade de estrelas nan: {df_kindle.filter(isnan(df_kindle.stars)).count()}')
print(f'Quantidade de análises nan: {df_kindle.filter(isnan(df_kindle.reviews)).count()}')
print(f'Quantidade de preços nan: {df_kindle.filter(isnan(df_kindle.price)).count()}')
print(f'Quantidade de livros serem KU nulos: {df_kindle.filter(df_kindle.isKindleUnlimited.isNull()).count()}')
print(f'Quantidade de categorias nan: {df_kindle.filter(isnan(df_kindle.category_id)).count()}')
print(f'Quantidade de livros serem de EP nulos: {df_kindle.filter(df_kindle.isEditorsPick.isNull()).count()}')
print(f'Quantidade de livros serem de GC nulos: {df_kindle.filter(df_kindle.isGoodReadsChoice.isNull()).count()}')
print(f'Quantidade de datas de publicação nulas: {df_kindle.filter(df_kindle.publishedDate.isNull()).count()}')
print(f'Quantidade de categorias nulas: {df_kindle.filter(df_kindle.category_name.isNull()).count()}')


> Modificar de nulo para um texto mais coerente

> Coluna _author_

In [0]:
df_kindle = df_kindle.withColumn('author',
                        when(df_kindle.author.isNull(), "Unknown")
                        .otherwise(df_kindle.author))

df_kindle.where(df_kindle.author == "Unknown").show(2)

> Coluna __soldBy

In [0]:
df_kindle = df_kindle.withColumn('soldBy',
                        when(df_kindle.soldBy.isNull(), "Unknown")
                        .otherwise(df_kindle.soldBy))

df_kindle.where(df_kindle.soldBy == "Unknown").show(2)

## Alterar a coluna _genre_ para não ter dados duplicados

In [0]:
df_goodreads = df_goodreads.withColumn('genre', split(df_goodreads.genre, ',')) \
                            .withColumn('genre', array_distinct(df_goodreads.genre))

## Salvar na camada _silver_

In [0]:
df_goodreads.write.format('delta').options(header=True).mode('overwrite').save('abfss://livros@recomendador.dfs.core.windows.net/silver/goodreads')
df_kindle.write.format('delta').options(header=True).mode('overwrite').save('abfss://livros@recomendador.dfs.core.windows.net/silver/kindle')

%md
## Exclusão de variáveis

In [0]:
del df_goodreads, df_kindle